In [31]:
from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.templates.default = "simple_white"

In [34]:
path = Path('data.csv')
df = pd.read_csv(path)
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [35]:
# Show distribution for most important columns when predicting Attrition True
# The feature importance CSV is included in the reporting directory
column_feature_top10 = df[['OverTime','JobRole','BusinessTravel','DistanceFromHome','MaritalStatus','NumCompaniesWorked','YearsSinceLastPromotion','Attrition']]

# Loop through every column in the dataframe and plot
'''
for i in column_feature_top10[0:]:
    fig = px.histogram(df,x=i)
    fig.show()
'''

'\nfor i in column_feature_top10[0:]:\n    fig = px.histogram(df,x=i)\n    fig.show()\n'

In [36]:
# over time and attrition
fig = px.histogram(column_feature_top10,x='OverTime',color='Attrition',barmode='group',title="Count of Over Time where Attrition is True")
fig.show()

In [39]:
# business travel and attrition
fig = px.histogram(column_feature_top10, x='BusinessTravel', color='Attrition', barmode='group', title='Level of Business Travel where Attrition is True')
fig.show()

In [38]:
# Business Travel and attrition
fig = px.histogram(column_feature_top10, x='BusinessTravel', color='Attrition', barmode='group', title='Level of Business Travel')
fig.show()

In [40]:
# Job Role and attrition
fig = px.histogram(column_feature_top10, x='JobRole', color='Attrition', barmode='group', title='Level of Business Travel')
fig.show()

In [37]:
# Show distribution for most important columns when predicting Attrition False
# The feature importance CSV is included in the reporting directory
column_feature_bottom10 = df[['JobLevel','WorkLifeBalance','JobInvolvement','EnvironmentSatisfaction','StockOptionLevel','JobRole']]
# Loop through every column in the dataframe and plot
"""for i in column_feature_bottom10[0:]:
    fig = px.histogram(df,x=i)
    fig.show()"""

'for i in column_feature_bottom10[0:]:\n    fig = px.histogram(df,x=i)\n    fig.show()'